In [5]:
!pip install git+https://github.com/suno-ai/bark.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install diffusers==0.23.0
!pip install gradio==3.50.2
!pip install --extra-index-url https://download.pytorch.org/whl/cu121;
!pip install torch==2.1.0
!pip install accelerate==0.24.0

  Cloning https://github.com/suno-ai/bark.git to /tmp/pip-req-build-sg7x7xsu
  Running command git clone --filter=blob:none --quiet https://github.com/suno-ai/bark.git /tmp/pip-req-build-sg7x7xsu
  Resolved https://github.com/suno-ai/bark.git to commit 773624d26db84278a55aacae9a16d7b25fbccab8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-bqwy1wo6
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-bqwy1wo6
  Resolved https://github.com/huggingface/transformers.git to commit 0ad4e7e6dad670a7151aaceb1af3c272a3bf73a8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: You must give at least one requirement to install (see "

In [6]:
from transformers import AutoProcessor, BarkModel
import scipy
import gradio as gr


In [ ]:
processor = AutoProcessor.from_pretrained("suno/bark")
model = BarkModel.from_pretrained("suno/bark")

def submit(prompt, voice, filename):
    if len(prompt) == 0:
        return {error_box: gr.Textbox(value="Enter prompt", visible=True)}
    #voice_preset = "v2/en_speaker_6"
    inputs = processor(prompt, voice_preset=voice)#_preset)

    audio_array = model.generate(**inputs)
    audio_array = audio_array.cpu().numpy().squeeze()

    sample_rate = model.generation_config.sample_rate
    scipy.io.wavfile.write(f"./{filename}", rate=sample_rate, data=audio_array)

    return {
        output_row: gr.Row(visible=True),
        text_out: "DONE",
        audio_out: f'./{filename}'
    }

with gr.Blocks() as demo:
    error_box = gr.Textbox(label="Error", visible=False)

    with gr.Row():
        drop_voices = gr.Dropdown(["v2/en_speaker_1", "v2/en_speaker_2", "v2/en_speaker_3", "v2/en_speaker_4", "v2/en_speaker_5", "v2/en_speaker_6", "v2/en_speaker_7", "v2/en_speaker_8", "v2/en_speaker_9"], label="voices 1-8 male/9 female", value=0)
        file_in = gr.Textbox(label="file", value="test.wav")
    with gr.Row():
        text_in = gr.Textbox(label="prompt")
    with gr.Row():
        submit_btn = gr.Button("Submit")
    with gr.Row(visible=True) as output_row:
        text_out = gr.Textbox(label="output")
        audio_out = gr.Audio(label="generated")
    submit_btn.click(
        submit,
        [text_in, drop_voices, file_in],
        [error_box, text_out, audio_out, output_row],
    )

demo.queue()
demo.launch(share=True, show_api=False, debug=True)